In [ ]:
# xls_to_csv_converter.ipynb

import pandas as pd
import io
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
import pyexcel as pe

# Création du widget d'upload
upload = widgets.FileUpload(accept='.xls', multiple=True)
display(upload)

# Fonction de nettoyage des noms de colonnes
def clean_column_name(name):
    if str(name).strip().lower() == "quantité":
        return "qte"
    return name

# Fonction de conversion à lancer à l'upload du fichier
def on_upload_change(change):
    if upload.value:
        for filedata in upload.value:
            filename = filedata['name']
            content = io.BytesIO(filedata['content'])

            try:
                # Lire le fichier .xls avec pyexcel
                book = pe.get_book(file_type="xls", file_stream=content)
                sheet = book[0]  # première feuille

                data = list(sheet.rows())
                if not data:
                    raise ValueError("Feuille vide")

                headers = data[0]
                rows = data[1:]

                df = pd.DataFrame(rows, columns=headers)

                # Renommer les colonnes si nécessaire
                df.columns = [clean_column_name(col) for col in df.columns]

                # Convertir les floats en int si applicable
                for col in df.columns:
                    df[col] = df[col].apply(lambda x: int(x) if isinstance(x, float) and x.is_integer() else x)

                # Enregistrer en CSV (ANSI / cp1252)
                output_path = Path(f"{Path(filename).stem}.csv")
                df.to_csv(output_path, index=False, sep=';', encoding='cp1252')
                print(f"\n✅ Fichier converti avec succès : {output_path}")

            except Exception as e:
                print(f"\n❌ Erreur lors de la lecture/conversion : {e}")

# Attacher l'événement
upload.observe(on_upload_change, names='value')